In [1]:
import torch
import torchvision

In [12]:
models = ['resnet18', 'resnet34','resnet50', 'resnet101','densenet121','mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small', 'vgg11', 'vgg13', 'squeezenet1_0', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'efficientnet_b0','efficientnet_b3', 'efficientnet_b4']

In [32]:
import os
folder_path = '/nfs/users/ext_shikhar.srivastava/workspace/TANS/training/logs/second_run/'

In [33]:
# get all subdirectories of folder_path
# get all .pt files in the subdirs 


In [27]:
num_classes = 15
for model in models:
    model_obj = getattr(torchvision.models, model)(pretrained=True)
    print(model_obj.__class__.__name__ )
    if model_obj.__class__.__name__ == 'ResNet':
        model_obj.fc = torch.nn.Linear(model_obj.fc.in_features, num_classes)
        model_obj.fc.weight.data.zero_()
        model_obj.fc.bias.data.zero_()
    
    elif ((model_obj.__class__.__name__ == 'MobileNetV2') | (model_obj.__class__.__name__ == 'EfficientNet') \
        | (model_obj.__class__.__name__ == 'MobileNetV3') | (model_obj.__class__.__name__ == 'VGG')  ):
        model_obj.classifier[-1] = torch.nn.Linear(model_obj.classifier[-1].in_features, num_classes)
        model_obj.classifier[-1].weight.data.zero_()
        model_obj.classifier[-1].bias.data.zero_()
    elif model_obj.__class__.__name__ == 'SqueezeNet':
        model_obj.classifier.Conv2d = torch.nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_obj.classifier.Conv2d.weight.data.zero_()
        model_obj.classifier.Conv2d.bias.data.zero_()   

    elif model_obj.__class__.__name__ == 'ShuffleNetV2':
        model_obj.fc = torch.nn.Linear(model_obj.fc.in_features, num_classes)
        model_obj.fc.weight.data.zero_()
        model_obj.fc.bias.data.zero_()
    else:
        #print(model_obj.__class__.__name__ )
        model_obj.classifier = torch.nn.Linear(model_obj.classifier.in_features, num_classes)
        model_obj.classifier.weight.data.zero_()
        model_obj.classifier.bias.data.zero_()


ResNet
ResNet
ResNet
ResNet
DenseNet
MobileNetV2
MobileNetV3
MobileNetV3
VGG
VGG
SqueezeNet
SqueezeNet
ShuffleNetV2


Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to /nfs/users/ext_shikhar.srivastava/.cache/torch/hub/checkpoints/shufflenetv2_x1-5666bf0f80.pth


  0%|          | 0.00/8.79M [00:00<?, ?B/s]

ShuffleNetV2


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /nfs/users/ext_shikhar.srivastava/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


...

  0%|          | 0.00/47.2M [00:00<?, ?B/s]

EfficientNet


Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-7eb33cd5.pth" to /nfs/users/ext_shikhar.srivastava/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-7eb33cd5.pth


  0%|          | 0.00/74.5M [00:00<?, ?B/s]

EfficientNet


In [28]:
model_obj

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(

In [19]:
from torchsummary import summary

In [23]:
summary(model_obj, (3, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 512, 1, 1]           --
|    └─Conv2d: 2-1                       [-1, 96, 11, 11]          14,208
|    └─ReLU: 2-2                         [-1, 96, 11, 11]          --
|    └─MaxPool2d: 2-3                    [-1, 96, 5, 5]            --
|    └─Fire: 2-4                         [-1, 128, 5, 5]           --
|    |    └─Conv2d: 3-1                  [-1, 16, 5, 5]            1,552
|    |    └─ReLU: 3-2                    [-1, 16, 5, 5]            --
|    |    └─Conv2d: 3-3                  [-1, 64, 5, 5]            1,088
|    |    └─ReLU: 3-4                    [-1, 64, 5, 5]            --
|    |    └─Conv2d: 3-5                  [-1, 64, 5, 5]            9,280
|    |    └─ReLU: 3-6                    [-1, 64, 5, 5]            --
|    └─Fire: 2-5                         [-1, 128, 5, 5]           --
|    |    └─Conv2d: 3-7                  [-1, 16, 5, 5]            2,064

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 512, 1, 1]           --
|    └─Conv2d: 2-1                       [-1, 96, 11, 11]          14,208
|    └─ReLU: 2-2                         [-1, 96, 11, 11]          --
|    └─MaxPool2d: 2-3                    [-1, 96, 5, 5]            --
|    └─Fire: 2-4                         [-1, 128, 5, 5]           --
|    |    └─Conv2d: 3-1                  [-1, 16, 5, 5]            1,552
|    |    └─ReLU: 3-2                    [-1, 16, 5, 5]            --
|    |    └─Conv2d: 3-3                  [-1, 64, 5, 5]            1,088
|    |    └─ReLU: 3-4                    [-1, 64, 5, 5]            --
|    |    └─Conv2d: 3-5                  [-1, 64, 5, 5]            9,280
|    |    └─ReLU: 3-6                    [-1, 64, 5, 5]            --
|    └─Fire: 2-5                         [-1, 128, 5, 5]           --
|    |    └─Conv2d: 3-7                  [-1, 16, 5, 5]            2,064

In [4]:
path = '/nfs/users/ext_shikhar.srivastava/workspace/TANS/training/logs/first_run/MNIST/shufflenet_v2_x0_5MNIST_shufflenet_v2_x0_5.pt'

In [5]:
model = torch.load(path, map_location=lambda storage, loc: storage)

In [6]:
m = torchvision.models.shufflenet_v2_x0_5(pretrained=False, num_classes=20)

In [8]:
model.state_dict

<bound method Module.state_dict of ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, moment

In [1]:
 import torch
 import torchvision
 
mnist = torchvision.datasets.MNIST('../data', train=True, download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(
                        (0.1307,), (0.3081,))]))


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [13]:
datasets = {'MNIST': torchvision.datasets.MNIST, 'FashionMNIST': torchvision.datasets.FashionMNIST, 'CIFAR10': torchvision.datasets.CIFAR10\
        , 'CIFAR100': torchvision.datasets.CIFAR100, 'STL10': torchvision.datasets.STL10, 'ImageNet': torchvision.datasets.ImageNet\
            , 'SVHN': torchvision.datasets.SVHN, 'KMNIST': torchvision.datasets.KMNIST, \
                'USPS': torchvision.datasets.USPS, 'QMNIST': torchvision.datasets.QMNIST}

In [1]:
import torchvision
da = torchvision.datasets.SVHN(root='../data', split='train', download=True, transform=torchvision.transforms.ToTensor())

Using downloaded and verified file: ../data/train_32x32.mat


In [2]:
set(da.labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [5]:
da = torchvision.datasets.USPS(root='../data', train = True, download=True, transform=torchvision.transforms.ToTensor())
len(set(da.targets))
#trainset = torchvision.datasets.USPS(root='./data', train=True, download=True)
#len(set(trainset.targets))

10

In [19]:
da = torchvision.datasets.QMNIST(root='../data', split='train', download=True,)

TypeError: __init__() got an unexpected keyword argument 'split'